In [329]:
import random
import os.path
import pandas as pd
import numpy as np

from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *

import matplotlib
matplotlib.style.use('ggplot')

In [ ]:
def exists(filepath):
    return os.path.exists(filepath)

# Split 48 million rows to 10 million(training) & 38 million(testing)

In [ ]:
if (exists('train10M.txt') and exists('test38M.txt')):
    print "train10M.txt and test38M.txt already created before."
else:
    !split -l 2000000 train.txt ff
    !cat ffaa ffat ffaf ffaq ffaj > train10M.txt
    !rm ffaa ffat ffaf ffaq ffaj
    !cat ff* > train38M.txt
    !rm ff*
    print "Newly created files: train10M.txt and test38M.txt."

# Split training data into 3 partitions

In [ ]:
if (exists('train5M.txt') and 
    exists('validation2M.txt') and 
    exists('test3M.txt')):
    print "train5M.txt, validation2M.txt, test3M.txt already created before."
else:
    !split -l 1000000 train10M.txt ff
    !cat ffaa ffaj ffad ffaf ffah > train5M.txt    
    !cat ffai ffae > validation2M.txt
    !cat ffac ffag ffab > test3M.txt
    !rm ff*
    print "Newly created files: train5M.txt, validation2M.txt, test3M.txt."

# Perform training and analysis with train5M data

In [404]:
df = pd.read_table('train5M.txt')
print df.shape

(4999999, 40)


# Remove and save the first column as y 

In [405]:
y = df['0']
df = df.drop('0', 1)

# Rename column for train5m data

In [407]:
new_col_names = ['f' + str(num) for num in range(1,len(df.columns)+1)]
df.columns = new_col_names

# Get summary stats for all integer columns

In [408]:
all_columns = list(df.columns)
integer_cols = all_columns[:13]
df[integer_cols].describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
count,2756813.000000,4999999.000000,3906401.000000,3853777.000000,4863896.000000,3887221.000000,4788996.000000,4997037.000000,4788996.000000,2756813.000000,4788996.000000,1177668.000000,3853777.000000
mean,3.550075,98.595417,30.839373,7.429061,18112.661813,118.854952,16.559919,12.785998,110.211554,0.610537,2.751485,0.946656,8.385096
std,9.865513,385.372407,494.089496,9.024720,68140.203950,421.572801,71.788627,19.683659,229.168662,0.690870,5.243586,5.745028,21.412443
min,0.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,2.000000,2.000000,561.000000,8.000000,1.000000,2.000000,11.000000,0.000000,1.000000,0.000000,2.000000
50%,1.000000,2.000000,6.000000,4.000000,2802.000000,33.000000,3.000000,7.000000,40.000000,1.000000,1.000000,0.000000,4.000000
75%,3.000000,34.000000,17.000000,10.000000,10082.000000,104.000000,12.000000,19.000000,114.000000,1.000000,3.000000,0.000000,10.000000
max,1900.000000,257675.000000,65535.000000,563.000000,23159456.000000,367553.000000,56311.000000,5064.000000,29019.000000,9.000000,165.000000,1881.000000,7393.000000


# Create files for each histogram

In [ ]:
def m_count_distinct(col):
    """
        count distinct values in all columns
        @param col: numpy 1D array
        @return: dictionary of value, freq
    """
    d = {}
    for x in list(col):
        if type(x) == np.float64 and np.isnan(x):
            d['NaN'] = d.get('NaN', 0) + 1
        else:
            d[x] = d.get(x, 0) + 1
    return d


# ==== create file from result of each freq count ===
for col in all_columns:
    filepath = 'histo-file/' + col + '.txt'
    if (exists(filepath)):
        print filepath + " already created."
    else:
        freq_count = m_count_distinct(df[col].values)
        with open(filepath, 'w') as f:
            for key, value in freq_count.items():
                row = str(key) + "," + str(value) + "\n"
                f.write(row)
        print "Newly created file:", filepath

# Plot histograms from stored files

In [ ]:
for col in all_columns:
    filepath = "histo-file/" + col + ".txt"
    histo_df = pd.read_csv(filepath, header=None, names=["val", "freq"])
    ax = histo_df.plot(x='val', y='freq', logy='True', title=col, xlim=(0))
    fig = ax.get_figure()
    fig.savefig('second-histo-img/' + col + '.png')

# Normalize integer columns after transpose

In [436]:
df_norm = df[integer_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df_norm.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,0.001053,0.000012,0.000671,0.001776,4.404249e-06,0.000022,0.000036,0.000395,0.000138,0.111111,0.006061,NaN,0.000541
1,0.001053,0.000012,0.000015,0.024867,3.311822e-05,0.000242,0.000071,0.000395,0.008443,0.111111,0.018182,0.001595,0.006087
2,NaN,0.003477,NaN,NaN,1.896418e-04,NaN,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
3,0.001579,0.000008,NaN,0.000000,8.635781e-08,0.000000,0.000053,0.000000,0.000000,0.111111,0.006061,NaN,0.000000
4,NaN,0.000008,NaN,NaN,5.537263e-04,NaN,0.000000,0.000000,0.000207,NaN,0.000000,NaN,NaN


# Select features to be used

In [461]:
cols_to_keep = all_columns[13:16]
cols_to_keep

['f14', 'f15', 'f16']

# One hot encoding

In [458]:
xx = df.iloc[:100000,:6]
print xx.shape
xx.head()

(100000, 6)


,f1,f2,f3,f4,f5,f6
0,2,0,44,1,102,8
1,2,0,1,14,767,89
2,NaN,893,NaN,NaN,4392,NaN
3,3,-1,NaN,0,2,0
4,NaN,-1,NaN,NaN,12824,NaN


In [484]:
xx = df.iloc[:10000,13:39]
oh = pd.get_dummies(xx)
print oh.shape

(10000, 43574)


In [459]:
#cols_to_keep = all_columns[:13] + all_columns[17:24]
df_onehot = encode_onehot(xx, ['f1', 'f2', 'f3', 'f4', 'f5'])
print df_onehot.shape
df_onehot.head()

/Users/fnokeke/dev/cs5304/ds-env/lib/python2.7/site-packages/ipykernel/__main__.py:20: FutureWarning: the 'outtype' keyword is deprecated, use 'orient' instead


(100000, 6)


,f6,f1,f2,f3,f4,f5
0,8,2,0,44,1,102
1,89,2,0,1,14,767
2,NaN,NaN,893,NaN,NaN,4392
3,0,3,-1,NaN,0,2
4,NaN,NaN,-1,NaN,NaN,12824


In [447]:
from sklearn.feature_extraction import DictVectorizer
 
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df


# Model and Evaluation

In [ ]:
X = df_onehot
y = test_data.index.values

### Logistic Regression

In [ ]:
from sklearn.linear_mode import LogisticRegression
model = LogisticRegression()
model.fit(X,y)

In [ ]:
model.score(X,y)

### Split up the data set into test and training sets

In [ ]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
testLogRModel = LogisticRegression()
testLogRModel.fit(x_train, y_train)

In [ ]:
testLogRModel.score(X,y)

### Predict class labels for the test set

In [ ]:
predicted = testLogRModel.predict(x_test)
print predicted

### AUC score from ROC

In [ ]:
from sklearn import metrics
print "accuracy:", metrics.accuracy_score(y_test, predicted)
# print "auc:", metrics.roc_auc_score(y_test, predicted)

### Perform cross validation using n folds

In [ ]:
from sklearn.cross_validation import cross_val_score
#n = 5
#scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy')
#print scores
#print scores.mean()

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_random_forest = RandomForestClassifier(n_estimators=10)
clf_random_forest.fit(X,y)

In [ ]:
clf_random_forest.score(X,y)